# Reported Earnings Per Share Data Scraper
The code below was used to obtain Earnings Per Share Data for a five year period, as was available on MarketWatch.com. 
The unique symbols of the companies from the 'SymbolsAndReportDates' file were used as inputs in a for loop to determine if there were available EPS data, and the available EPS data saved to an Excel file, 'ReportedEPS', having 2 columns, 'EPS', representing the EPS for available symbols, and 'YEAR', representing the years for which the EPS was available.

In [ ]:
url_formatted = 'http://www.marketwatch.com/investing/stock/%s/financials'
eqList = df.Equities.unique()
eEPS = {}

# Navigate to website for each unique ticker
for ticker in eqList:
    try:
        res = requests.get(url_formatted % ticker)
    except:
        EPS = "N/A"
        YEAR = "N/A"
        continue
    else:
        res.raise_for_status()
        epsSoup = bs4.BeautifulSoup(res.text,"html5lib")
        
        try:
            #Find the closest link with id value and go to previous sibling for earnings data
            EPS = epsSoup.find('tr', id="ratio_Eps1YrAnnualGrowth").find_previous_sibling("tr").find_all('td', class_="valueCell")
            YEAR = epsSoup.find("tr", class_="topRow").find_all('th',string=re.compile("\d{4}"))
        except:
            EPS = "N/A"
            YEAR = "N/A"
            continue
            
        # Extract 2012-2016 EPS data and save to list EarnList 4 times for each quarter's earnings
        epsPerYear = {}
        for yearEle, earnEle in zip(YEAR, EPS):
            epsPerYear[yearEle.get_text()] = earnEle.get_text()
        eEPS[ticker] = epsPerYear